<a href="https://colab.research.google.com/github/Vigneshthanga/258-Deep-Learning/blob/master/Ungraded5/QuickStartTF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Tensorflow libraries

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

## Loading MNIST data

In [0]:
mnist = tf.keras.datasets.mnist
(xtrain, ytrain), (xtest, ytest) = mnist.load_data()
xtrain = xtrain / 255.0
xtest = xtest / 255.0
xtrain = xtrain[..., tf.newaxis]
xtest = xtest[..., tf.newaxis]

## Train and test data split

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (xtrain, ytrain)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((xtest, ytest)).batch(32)

## Model Definition

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)
model = MyModel()

## Loss function and optimizer

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Defining Train step function using tf.function decorator

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

## Defining Test step function using tf.function decorator

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

## Model Training

In [13]:
EPOCHS = 5
for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.12988385558128357, Accuracy: 96.09500122070312, Test Loss: 0.06212998181581497, Test Accuracy: 97.87999725341797
Epoch 2, Loss: 0.04100639000535011, Accuracy: 98.74166870117188, Test Loss: 0.055458951741456985, Test Accuracy: 98.16999816894531
Epoch 3, Loss: 0.02126135118305683, Accuracy: 99.33833312988281, Test Loss: 0.05533996596932411, Test Accuracy: 98.23999786376953
Epoch 4, Loss: 0.012884525582194328, Accuracy: 99.57500457763672, Test Loss: 0.06049220263957977, Test Accuracy: 98.0